# Building POS Tagger using Keras Library without considering PAD values

We are going to work with Twitter *Hindi-English* code mixed tweets. For the purpose of data, we have 1981 tweets which are in **conll** format. They are tagged manually with the *language* and *POS*

#### Load Dependencies

In [1]:
import keras
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Embedding # new!
from keras.layers import Conv1D, SpatialDropout1D, GlobalMaxPooling1D

from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint # new! 
from sklearn.model_selection import train_test_split
import os # new! 
from sklearn.metrics import roc_auc_score, roc_curve # new!
import pandas as pd
import matplotlib.pyplot as plt # new!
%matplotlib inline

Using TensorFlow backend.


In [2]:
from keras import backend as K
 
def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
 
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy

In [3]:
# function to define arr with (word, lang, tag)
def make_arr(f1):
    twitter_file = open(f1, "r")
    sentences = []
    sent = []
    for line in twitter_file:
        temp = line.split('\t')
        
        if temp[0] == '\n':
            sentences.append(sent)
            sent = []
            continue

        check = list(temp[2])
        if '\n' in check:
            check.remove('\n')

        temp[2] = ''.join(check)
        sent.append((temp[0], temp[1], temp[2]))


    return sentences

#### Load data 

In [4]:
tagged_sentences = make_arr("Twitter_file.txt")
no_of_sentences = 1981 # number of sentences to take for corpus

In [5]:
len(tagged_sentences)

1981

#### Separate the sentences words and tags into two different arrays

In [6]:
sentences, sentence_tags = [], []
for sent in tagged_sentences:
    sentence, lang, tags = zip(*sent)
    sentences.append(np.array(sentence))
    sentence_tags.append(np.array(tags))

In [7]:
len(sentences), len(sentence_tags)

(1981, 1981)

In [8]:
sentences[0], sentence_tags[0]

(array(['I', 'will', 'read', 'kal', 'pakka', '.', 'Have', 'to', 'study',
        'right', 'now', 'okay', '?', '\xf0\x9f\x98\x80',
        'https://t.co/C2SrZhfJfK'], dtype='|S23'),
 array(['PR_PRP', 'V_VAUX', 'V_VM', 'RB_ALC', 'JJ', 'RD_PUNC', 'V_VM',
        'RP_RPD', 'N_NN', 'RB_AMN', 'RB_AMN', 'RP_INJ', 'RD_PUNC', 'E',
        'U'], dtype='|S7'))

In [9]:
len(sentences[0]), len(sentence_tags[0])

(15, 15)

### Train the required number of sentences *

In [10]:
sentences = sentences[:no_of_sentences]
sentence_tags = sentence_tags[:no_of_sentences]

In [11]:
(train_sentences, test_sentences, train_tags, test_tags) = train_test_split(sentences, sentence_tags, test_size=0.2)

In [12]:
print sentences[0], sentence_tags[0]

['I' 'will' 'read' 'kal' 'pakka' '.' 'Have' 'to' 'study' 'right' 'now'
 'okay' '?' '\xf0\x9f\x98\x80' 'https://t.co/C2SrZhfJfK'] ['PR_PRP' 'V_VAUX' 'V_VM' 'RB_ALC' 'JJ' 'RD_PUNC' 'V_VM' 'RP_RPD' 'N_NN'
 'RB_AMN' 'RB_AMN' 'RP_INJ' 'RD_PUNC' 'E' 'U']


#### For the embedding layer to work, find the count of unique words

In [13]:
words, tags = set([]), set([])
 
for s in train_sentences:
    for w in s:
        words.add(w.lower())

for ts in train_tags:
    for t in ts:
        tags.add(t)
        
word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs
 
tag2index = {t: i + 1 for i, t in enumerate(list(tags))}
tag2index['-PAD-'] = 0  # The special value used to padding

In [14]:
print train_tags[0]

['@' 'DT' 'JJ' 'RP_INJ' 'PR_PRL' 'V_VM' 'V_VM' 'PR_PRP' 'V_VM' 'PR_PRP'
 'DM_DMD' 'V_VM' 'PR_PRP' 'V_VAUX' 'PR_PRL' 'V_VM' 'N_NN' 'RP_NEG'
 'DM_DMD' 'PR_PRP' 'RD_RDF' 'V_VAUX' 'JJ' 'RD_RDF' 'N_NN' 'V_VM' 'V_VAUX']


#### Convert the words to the numpy array so that, numerical data can be used for training

In [15]:
train_sentences_X, test_sentences_X, train_tags_y, test_tags_y = [], [], [], []
 
for s in train_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sentences_X.append(s_int)

for s in test_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sentences_X.append(s_int)

for s in train_tags:
    train_tags_y.append([tag2index[t] for t in s])

for s in test_tags:
    test_tags_y.append([tag2index[t] for t in s])

print(train_sentences_X[0])
print(train_tags_y[0])
print(test_sentences_X[0])
print(test_tags_y[0])
print len(train_sentences_X[0]), len((train_tags_y[0]))

[200, 3353, 6999, 5321, 5260, 7838, 3503, 2334, 8545, 8350, 3588, 4892, 8302, 6433, 5260, 2593, 1352, 7060, 3588, 8350, 6920, 6433, 2947, 3815, 2397, 7259, 6433]
[18, 7, 6, 27, 28, 4, 4, 39, 4, 39, 33, 4, 39, 29, 28, 4, 17, 8, 33, 39, 9, 29, 6, 9, 17, 4, 29]
[1, 350, 2319, 2282, 2576, 4475, 350, 8464, 2465, 736, 584, 6645, 1, 1, 350, 6546, 1, 6545, 4215, 2448, 350, 8006, 350, 1, 2282, 7178, 350]
[17, 32, 15, 36, 7, 17, 32, 29, 17, 19, 36, 7, 17, 17, 32, 32, 29, 32, 39, 29, 32, 6, 32, 4, 36, 17, 32]
27 27


#### Pad the sequences because Keras can only work with fixed size sequences

In [16]:
MAX_LENGTH = len(max(train_sentences_X, key=len))
print(MAX_LENGTH)  

93


In [17]:
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')
 
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[ 200 3353 6999 5321 5260 7838 3503 2334 8545 8350 3588 4892 8302 6433
 5260 2593 1352 7060 3588 8350 6920 6433 2947 3815 2397 7259 6433    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0]
[   1  350 2319 2282 2576 4475  350 8464 2465  736  584 6645    1    1
  350 6546    1 6545 4215 2448  350 8006  350    1 2282 7178  350    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0]
[18  7  6 27 28  4  4 39  4 39 33  4 39 29 28  4 17  8

In [18]:
print(len(tags))

39


#### Define our neural architecture

In [20]:
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy', ignore_class_accuracy(0)])

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 93, 128)           1114624   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 93, 512)           788480    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 93, 512)           1574912   
_________________________________________________________________
time_distributed_2 (TimeDist (None, 93, 40)            20520     
_________________________________________________________________
activation_1 (Activation)    (None, 93, 40)            0         
Total params: 3,498,536
Trainable params: 3,498,536
Non-trainable params: 0
_________________________________________________________________


#### Since we have 39 tags for each word we need to convert it to ONE HOT ENCODING

In [22]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [23]:
cat_train_tags_y = to_categorical(train_tags_y, len(tag2index))
print cat_train_tags_y[0]

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [ ]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=128, epochs=1000, validation_split=0.2)

Train on 1267 samples, validate on 317 samples
Epoch 1/1000
1267/1267 [==============================] - 45s 36ms/step - loss: 1.8366 - acc: 0.6957 - ignore_accuracy: 0.0000e+00 - val_loss: 0.8427 - val_acc: 0.7768 - val_ignore_accuracy: 0.0000e+00
Epoch 2/1000
1267/1267 [==============================] - 39s 31ms/step - loss: 0.8346 - acc: 0.7848 - ignore_accuracy: 0.0000e+00 - val_loss: 0.7720 - val_acc: 0.7857 - val_ignore_accuracy: 0.0000e+00
Epoch 3/1000
1267/1267 [==============================] - 39s 30ms/step - loss: 0.7365 - acc: 0.8029 - ignore_accuracy: 0.0000e+00 - val_loss: 0.6973 - val_acc: 0.8112 - val_ignore_accuracy: 0.0000e+00
Epoch 4/1000
1267/1267 [==============================] - 39s 30ms/step - loss: 0.6919 - acc: 0.8054 - ignore_accuracy: 0.0000e+00 - val_loss: 0.6638 - val_acc: 0.8095 - val_ignore_accuracy: 0.0000e+00
Epoch 5/1000
1267/1267 [==============================] - 42s 33ms/step - loss: 0.6695 - acc: 0.8085 - ignore_accuracy: 0.0000e+00 - val_loss: 0.

Epoch 82/1000
1267/1267 [==============================] - 39s 30ms/step - loss: 0.0025 - acc: 0.9999 - ignore_accuracy: 0.5051 - val_loss: 0.4908 - val_acc: 0.9334 - val_ignore_accuracy: 0.0000e+00
Epoch 83/1000
1267/1267 [==============================] - 40s 31ms/step - loss: 0.0024 - acc: 0.9999 - ignore_accuracy: 0.4949 - val_loss: 0.4925 - val_acc: 0.9337 - val_ignore_accuracy: 0.0000e+00
Epoch 84/1000
1267/1267 [==============================] - 39s 31ms/step - loss: 0.0023 - acc: 0.9999 - ignore_accuracy: 0.4041 - val_loss: 0.4934 - val_acc: 0.9334 - val_ignore_accuracy: 0.0000e+00
Epoch 85/1000
1267/1267 [==============================] - 39s 31ms/step - loss: 0.0022 - acc: 0.9999 - ignore_accuracy: 0.6062 - val_loss: 0.4959 - val_acc: 0.9332 - val_ignore_accuracy: 0.0000e+00
Epoch 86/1000
1267/1267 [==============================] - 39s 31ms/step - loss: 0.0021 - acc: 1.0000 - ignore_accuracy: 0.5051 - val_loss: 0.4978 - val_acc: 0.9336 - val_ignore_accuracy: 0.0000e+00
Epoch

1267/1267 [==============================] - 39s 31ms/step - loss: 0.0011 - acc: 1.0000 - ignore_accuracy: 0.6062 - val_loss: 0.5359 - val_acc: 0.9339 - val_ignore_accuracy: 0.0000e+00
Epoch 124/1000
1267/1267 [==============================] - 39s 31ms/step - loss: 0.0011 - acc: 1.0000 - ignore_accuracy: 0.5959 - val_loss: 0.5372 - val_acc: 0.9339 - val_ignore_accuracy: 0.0000e+00
Epoch 125/1000
1267/1267 [==============================] - 39s 30ms/step - loss: 0.0011 - acc: 1.0000 - ignore_accuracy: 0.4949 - val_loss: 0.5378 - val_acc: 0.9338 - val_ignore_accuracy: 0.0000e+00
Epoch 126/1000
1267/1267 [==============================] - 39s 31ms/step - loss: 0.0011 - acc: 1.0000 - ignore_accuracy: 0.6062 - val_loss: 0.5392 - val_acc: 0.9338 - val_ignore_accuracy: 0.0000e+00
Epoch 127/1000
1267/1267 [==============================] - 39s 31ms/step - loss: 0.0011 - acc: 1.0000 - ignore_accuracy: 0.6062 - val_loss: 0.5397 - val_acc: 0.9337 - val_ignore_accuracy: 0.0000e+00
Epoch 128/1000


1267/1267 [==============================] - 53s 42ms/step - loss: 8.0113e-04 - acc: 1.0000 - ignore_accuracy: 0.5959 - val_loss: 0.5768 - val_acc: 0.9337 - val_ignore_accuracy: 0.0000e+00
Epoch 204/1000
1267/1267 [==============================] - 54s 42ms/step - loss: 7.9964e-04 - acc: 1.0000 - ignore_accuracy: 0.5959 - val_loss: 0.5773 - val_acc: 0.9336 - val_ignore_accuracy: 0.0000e+00
Epoch 205/1000
1267/1267 [==============================] - 54s 42ms/step - loss: 7.9820e-04 - acc: 1.0000 - ignore_accuracy: 0.4949 - val_loss: 0.5773 - val_acc: 0.9339 - val_ignore_accuracy: 0.0000e+00
Epoch 206/1000
1267/1267 [==============================] - 53s 42ms/step - loss: 7.9679e-04 - acc: 1.0000 - ignore_accuracy: 0.6062 - val_loss: 0.5775 - val_acc: 0.9336 - val_ignore_accuracy: 0.0000e+00
Epoch 207/1000
1267/1267 [==============================] - 53s 42ms/step - loss: 7.9545e-04 - acc: 1.0000 - ignore_accuracy: 0.5959 - val_loss: 0.5775 - val_acc: 0.9337 - val_ignore_accuracy: 0.0000

In [24]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=128, epochs=1000, validation_split=0.2)

Train on 1267 samples, validate on 317 samples
Epoch 1/1000
1267/1267 [==============================] - 45s 36ms/step - loss: 1.8366 - acc: 0.6957 - ignore_accuracy: 0.0000e+00 - val_loss: 0.8427 - val_acc: 0.7768 - val_ignore_accuracy: 0.0000e+00
Epoch 2/1000
1267/1267 [==============================] - 39s 31ms/step - loss: 0.8346 - acc: 0.7848 - ignore_accuracy: 0.0000e+00 - val_loss: 0.7720 - val_acc: 0.7857 - val_ignore_accuracy: 0.0000e+00
Epoch 3/1000
1267/1267 [==============================] - 39s 30ms/step - loss: 0.7365 - acc: 0.8029 - ignore_accuracy: 0.0000e+00 - val_loss: 0.6973 - val_acc: 0.8112 - val_ignore_accuracy: 0.0000e+00
Epoch 4/1000
1267/1267 [==============================] - 39s 30ms/step - loss: 0.6919 - acc: 0.8054 - ignore_accuracy: 0.0000e+00 - val_loss: 0.6638 - val_acc: 0.8095 - val_ignore_accuracy: 0.0000e+00
Epoch 5/1000
1267/1267 [==============================] - 42s 33ms/step - loss: 0.6695 - acc: 0.8085 - ignore_accuracy: 0.0000e+00 - val_loss: 0.

1267/1267 [==============================] - 39s 30ms/step - loss: 0.0788 - acc: 0.9833 - ignore_accuracy: 0.0000e+00 - val_loss: 0.3215 - val_acc: 0.9343 - val_ignore_accuracy: 0.0000e+00
Epoch 42/1000
1267/1267 [==============================] - 39s 31ms/step - loss: 0.0714 - acc: 0.9849 - ignore_accuracy: 0.0000e+00 - val_loss: 0.3274 - val_acc: 0.9348 - val_ignore_accuracy: 0.0000e+00
Epoch 43/1000
1267/1267 [==============================] - 39s 30ms/step - loss: 0.0651 - acc: 0.9865 - ignore_accuracy: 0.0000e+00 - val_loss: 0.3381 - val_acc: 0.9343 - val_ignore_accuracy: 0.0000e+00
Epoch 44/1000
1267/1267 [==============================] - 39s 31ms/step - loss: 0.0592 - acc: 0.9876 - ignore_accuracy: 0.0000e+00 - val_loss: 0.3394 - val_acc: 0.9344 - val_ignore_accuracy: 0.0000e+00
Epoch 45/1000
1267/1267 [==============================] - 39s 30ms/step - loss: 0.0539 - acc: 0.9889 - ignore_accuracy: 0.0000e+00 - val_loss: 0.3416 - val_acc: 0.9357 - val_ignore_accuracy: 0.0000e+00

Epoch 82/1000
1267/1267 [==============================] - 39s 30ms/step - loss: 0.0025 - acc: 0.9999 - ignore_accuracy: 0.5051 - val_loss: 0.4908 - val_acc: 0.9334 - val_ignore_accuracy: 0.0000e+00
Epoch 83/1000
1267/1267 [==============================] - 40s 31ms/step - loss: 0.0024 - acc: 0.9999 - ignore_accuracy: 0.4949 - val_loss: 0.4925 - val_acc: 0.9337 - val_ignore_accuracy: 0.0000e+00
Epoch 84/1000
1267/1267 [==============================] - 39s 31ms/step - loss: 0.0023 - acc: 0.9999 - ignore_accuracy: 0.4041 - val_loss: 0.4934 - val_acc: 0.9334 - val_ignore_accuracy: 0.0000e+00
Epoch 85/1000
1267/1267 [==============================] - 39s 31ms/step - loss: 0.0022 - acc: 0.9999 - ignore_accuracy: 0.6062 - val_loss: 0.4959 - val_acc: 0.9332 - val_ignore_accuracy: 0.0000e+00
Epoch 86/1000
1267/1267 [==============================] - 39s 31ms/step - loss: 0.0021 - acc: 1.0000 - ignore_accuracy: 0.5051 - val_loss: 0.4978 - val_acc: 0.9336 - val_ignore_accuracy: 0.0000e+00
Epoch

1267/1267 [==============================] - 39s 31ms/step - loss: 0.0011 - acc: 1.0000 - ignore_accuracy: 0.6062 - val_loss: 0.5359 - val_acc: 0.9339 - val_ignore_accuracy: 0.0000e+00
Epoch 124/1000
1267/1267 [==============================] - 39s 31ms/step - loss: 0.0011 - acc: 1.0000 - ignore_accuracy: 0.5959 - val_loss: 0.5372 - val_acc: 0.9339 - val_ignore_accuracy: 0.0000e+00
Epoch 125/1000
1267/1267 [==============================] - 39s 30ms/step - loss: 0.0011 - acc: 1.0000 - ignore_accuracy: 0.4949 - val_loss: 0.5378 - val_acc: 0.9338 - val_ignore_accuracy: 0.0000e+00
Epoch 126/1000
1267/1267 [==============================] - 39s 31ms/step - loss: 0.0011 - acc: 1.0000 - ignore_accuracy: 0.6062 - val_loss: 0.5392 - val_acc: 0.9338 - val_ignore_accuracy: 0.0000e+00
Epoch 127/1000
1267/1267 [==============================] - 39s 31ms/step - loss: 0.0011 - acc: 1.0000 - ignore_accuracy: 0.6062 - val_loss: 0.5397 - val_acc: 0.9337 - val_ignore_accuracy: 0.0000e+00
Epoch 128/1000


1267/1267 [==============================] - 44s 35ms/step - loss: 8.9060e-04 - acc: 1.0000 - ignore_accuracy: 0.6969 - val_loss: 0.5607 - val_acc: 0.9336 - val_ignore_accuracy: 0.0000e+00
Epoch 164/1000
1267/1267 [==============================] - 46s 36ms/step - loss: 8.8709e-04 - acc: 1.0000 - ignore_accuracy: 0.6062 - val_loss: 0.5599 - val_acc: 0.9338 - val_ignore_accuracy: 0.0000e+00
Epoch 165/1000
1267/1267 [==============================] - 42s 33ms/step - loss: 8.8247e-04 - acc: 1.0000 - ignore_accuracy: 0.5959 - val_loss: 0.5614 - val_acc: 0.9340 - val_ignore_accuracy: 0.0000e+00
Epoch 166/1000
1267/1267 [==============================] - 46s 37ms/step - loss: 8.7919e-04 - acc: 1.0000 - ignore_accuracy: 0.6969 - val_loss: 0.5611 - val_acc: 0.9340 - val_ignore_accuracy: 0.0000e+00
Epoch 167/1000
1267/1267 [==============================] - 45s 36ms/step - loss: 8.7580e-04 - acc: 1.0000 - ignore_accuracy: 0.6062 - val_loss: 0.5618 - val_acc: 0.9339 - val_ignore_accuracy: 0.0000

1267/1267 [==============================] - 53s 42ms/step - loss: 8.0113e-04 - acc: 1.0000 - ignore_accuracy: 0.5959 - val_loss: 0.5768 - val_acc: 0.9337 - val_ignore_accuracy: 0.0000e+00
Epoch 204/1000
1267/1267 [==============================] - 54s 42ms/step - loss: 7.9964e-04 - acc: 1.0000 - ignore_accuracy: 0.5959 - val_loss: 0.5773 - val_acc: 0.9336 - val_ignore_accuracy: 0.0000e+00
Epoch 205/1000
1267/1267 [==============================] - 54s 42ms/step - loss: 7.9820e-04 - acc: 1.0000 - ignore_accuracy: 0.4949 - val_loss: 0.5773 - val_acc: 0.9339 - val_ignore_accuracy: 0.0000e+00
Epoch 206/1000
1267/1267 [==============================] - 53s 42ms/step - loss: 7.9679e-04 - acc: 1.0000 - ignore_accuracy: 0.6062 - val_loss: 0.5775 - val_acc: 0.9336 - val_ignore_accuracy: 0.0000e+00
Epoch 207/1000
1267/1267 [==============================] - 53s 42ms/step - loss: 7.9545e-04 - acc: 1.0000 - ignore_accuracy: 0.5959 - val_loss: 0.5775 - val_acc: 0.9337 - val_ignore_accuracy: 0.0000

KeyboardInterrupt: 

In [25]:
scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))
print ({model.metrics_names[1]},  {scores[1] * 100})

397/397 [==============================] - 4s 10ms/step
(set(['acc']), set([93.18003271628987]))


#### Convert back the categorical to tokens

In [26]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences

In [28]:
predictions = model.predict(test_sentences_X)

print len(predictions[0])
print len(test_tags_y[0])

# print first few predictions
token_sequences = logits_to_tokens(predictions, {i: t for t, i in tag2index.items()})
# print(logits_to_tokens(predictions[:2], {i: t for t, i in tag2index.items()}))
test_tags_sequences = logits_to_tokens(test_tags_y, {i: t for t, i in tag2index.items()})

print len(predictions), len(test_tags_sequences)
# token_sequences[0] = filter(lambda a: a != '-PAD-', token_sequences[0])
# print (test_sentences[0])
print (token_sequences[0][:len(test_tags[0])])
print (test_tags[0])

93
93
397 397
['JJ', 'RD_PUNC', 'RB_AMN', 'PSP', 'DT', 'N_NN', 'RD_PUNC', 'V_VAUX', 'N_NN', 'QT_QTO', 'PSP', 'DT', 'N_NN', 'N_NN', 'RD_PUNC', 'RD_PUNC', 'N_NN', 'RD_PUNC', 'PR_PRP', 'V_VAUX', 'RD_PUNC', 'N_NN', 'RD_PUNC', 'N_NN', 'PSP', 'N_NN', 'RD_PUNC']
['N_NN' 'RD_PUNC' 'N_NST' 'PSP' 'DT' 'N_NN' 'RD_PUNC' 'V_VAUX' 'N_NN'
 'QT_QTO' 'PSP' 'DT' 'N_NN' 'N_NN' 'RD_PUNC' 'RD_PUNC' 'V_VAUX' 'RD_PUNC'
 'PR_PRP' 'V_VAUX' 'RD_PUNC' 'JJ' 'RD_PUNC' 'V_VM' 'PSP' 'N_NN' 'RD_PUNC']
